In [21]:
import numpy as np
import pandas as pd

In [22]:
df = pd.read_csv('dataset.csv') 

In [23]:
df['Timestamp'] = pd.to_datetime(
    df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
    dayfirst=True
)

In [24]:
#Map traffic levels to numeric
traffic_map = {'low': 0.5, 'medium': 1.5, 'average': 2.0, 'high': 3.0}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

In [25]:
#Map vehicle type to weights
vehicle_map = {'cycle': 0.5, 'car': 1.5, 'bike': 1.0, 'truck': 2.0}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_map)

In [26]:
# Calculate occupancy rate
df['OccupancyRate'] = df['Occupancy'] / df['Capacity']
df['IsSpecialDay'] = df['IsSpecialDay'].astype(int) 

In [27]:
df['BasePrice'] = 10  # or any base value you prefer

alpha = 0.5
theta = 2
beta = 0.2
gamma = 0.3
eta = 1.5
delta = 0.5
epsilon = 0.2

In [28]:
def calculate_raw_demand(row):
    occupancy_effect = alpha * (row['OccupancyRate'] ** theta)
    queue_effect = beta * np.log(1 + row['QueueLength'])
    traffic_effect = gamma * (row['TrafficLevel'] ** eta)
    special_day_effect = delta * row['IsSpecialDay']
    vehicle_effect = epsilon * row['VehicleWeight']

    raw_demand = occupancy_effect + queue_effect + traffic_effect + special_day_effect + vehicle_effect

    return raw_demand

#Nonlinear terms (square, log) let you adjust how sharply prices rise when factors spike.


In [29]:
df['RawDemand'] = df.apply(calculate_raw_demand, axis=1)

In [30]:
from scipy.special import expit 
df['PriceMultiplier'] = 0.5 + 1.5 * expit(df['RawDemand'])

In [31]:
df['Price'] = df['BasePrice'] * df['PriceMultiplier']

In [32]:
print(df[['RawDemand', 'PriceMultiplier', 'Price']].head())

   RawDemand  PriceMultiplier      Price
0   0.550284         1.451302  14.513021
1   0.550847         1.451498  14.514981
2   0.635400         1.480569  14.805694
3   0.642983         1.483141  14.831411
4   0.559579         1.454533  14.545328


In [35]:
lot_df = df.groupby('SystemCodeNumber').agg({
    'Latitude': 'mean',
    'Longitude': 'mean',
    'Occupancy': 'mean',
    'Capacity': 'mean',
    'Price': 'mean'
}).reset_index()

In [38]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return 2 * R * np.arcsin(np.sqrt(a))

In [39]:
coords = lot_df[['Latitude', 'Longitude']].values
dist_matrix = np.array([
    [haversine(lat1, lon1, lat2, lon2) for lat2, lon2 in coords]
    for lat1, lon1 in coords
])

In [40]:
adjusted_prices = []

In [41]:
for i in range(len(lot_df)):
    lot = lot_df.iloc[i]
    distances = dist_matrix[i]
    
    # Find nearby lots within 0.5 km (excluding self)
    nearby_idx = np.where((distances < 0.5) & (distances > 0))[0]
    nearby_prices = lot_df.iloc[nearby_idx]['Price'] if len(nearby_idx) > 0 else pd.Series(dtype='float64')
    
    price = lot['Price']
    
    # Case 1: Lot is full (or nearly full) and cheaper lots are nearby
    if lot['OccupancyRate'] >= 0.95 and not nearby_prices.empty and (nearby_prices < price).any():
        price = nearby_prices.mean() * 0.95  # Reduce price slightly
    
    # Case 2: All nearby lots are more expensive → increase price up to 10%
    elif not nearby_prices.empty and (nearby_prices > price).all():
        price = min(price * 1.10, nearby_prices.mean())  # Conservative increase
    
    adjusted_prices.append(price)

In [42]:
lot_df['AdjustedPrice'] = adjusted_prices

In [43]:
print(lot_df[['SystemCodeNumber', 'Price', 'AdjustedPrice', 'OccupancyRate']].head())

  SystemCodeNumber      Price  AdjustedPrice  OccupancyRate
0      BHMBCCMKT01  16.651024      17.228261       0.280814
1      BHMBCCTHL01  17.228261      17.228261       0.745110
2      BHMEURBRD01  17.107921      17.107921       0.643602
3      BHMMBMMBX01  17.189923      17.189923       0.694762
4      BHMNCPHST01  16.916888      17.003838       0.464739


In [46]:
final_df = lot_df[['SystemCodeNumber', 'Latitude', 'Longitude', 'Occupancy', 'Capacity',
                   'OccupancyRate', 'Price', 'AdjustedPrice']]

In [47]:
final_df.to_csv('competitive_pricing_output.csv', index=False)
print("✅ Final CSV file 'competitive_pricing_output.csv' has been saved.")

✅ Final CSV file 'competitive_pricing_output.csv' has been saved.
